In [11]:
import torch
import qpth
import cvxpy as cp
import numpy as np

In [85]:
# from https://www.cvxpy.org/examples/basic/quadratic_program.html
# arrays come from https://cvxopt.org/examples/tutorial/qp.html

n = 2
m = 2

Q = 2*np.array([ [2, .5], [.5, 1] ])
p = np.array([1.0, 1.0])

G = np.array([[-1.0,0.0],[0.0,-1.0]])
h = np.array([0.0,0.0])

#A = np.array([1.0, 1.0], (1,2))
A = np.array([1.0, 1.0]).reshape((1,2))
b = np.array(1.0)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, Q) + p.T@x),
                 [G@x <= h,
                  A@x == b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)



The optimal value is 1.875
A solution x is
[0.25 0.75]
A dual solution corresponding to the inequality constraints is
[0. 0.]


In [86]:
res = qpth.qp.QPFunction(verbose=False)(*[
    torch.tensor(v).float()
    for v in [Q, p, G, h, A, b]
])
print(xsol, res)

[0, 0] tensor([[0.2500, 0.7500]])


In [87]:
from torch.autograd import Function, Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F

class OptNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.A = Parameter(torch.rand((1,2)).double())
        self.Q = Variable(2*torch.tensor([ [2, .5], [.5, 1] ]).double())
        self.p = Variable(torch.tensor([1.0, 1.0]).double())
        self.G = Variable(torch.tensor([[-1.0,0.0],[0.0,-1.0]]).double())
        self.h = Variable(torch.tensor([0.0,0.0]).double())
        self.b = Variable(torch.tensor(1.0).double())

    def forward(self, data):
        return qpth.qp.QPFunction(verbose=-1)(
            self.Q, self.p, self.G, self.h, self.A, self.b
        ).float()#.view_as(puzzles)


In [88]:
model = OptNet()
model.forward(None)

tensor([[0.2549, 1.1306]], grad_fn=<CopyBackwards>)

In [96]:
model = OptNet()
print('Initial value', model.A)

loss_fn = torch.nn.MSELoss()
learning_rate = 1e-1
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

progress = 25
for t in range(200):
    y_true = torch.tensor([0.25, 0.75]) # should be [1, 1]
    #y_true = torch.tensor([[0.45454545, 0.31818182]]) # should be [1.5, 1]
    y_pred = model(None)
    loss = loss_fn(y_pred, y_true)
    if (t+1)%progress == 0:
        print('Iteration {}, loss = {}'.format(t, loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print('Fitted value', model.A)

Initial value Parameter containing:
tensor([[0.8195, 0.4173]], dtype=torch.float64, requires_grad=True)
Iteration 24, loss = 0.008166614919900894
Iteration 49, loss = 0.0021558504085987806
Iteration 74, loss = 0.00010271537030348554
Iteration 99, loss = 2.5335580744467734e-07
Iteration 124, loss = 2.3296068718536844e-07
Iteration 149, loss = 3.812014881532377e-08
Iteration 174, loss = 3.6944207693068165e-09
Iteration 199, loss = 2.96603630545178e-10
Fitted value Parameter containing:
tensor([[1.0000, 1.0000]], dtype=torch.float64, requires_grad=True)
